In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import preprocessing
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heuristics-dl-interstellarwar/Sample_Submission.csv
/kaggle/input/heuristics-dl-interstellarwar/train.csv
/kaggle/input/heuristics-dl-interstellarwar/test.csv


In [2]:
sample_sub = pd.read_csv('../input/heuristics-dl-interstellarwar/Sample_Submission.csv')
x_train_data = pd.read_csv('../input/heuristics-dl-interstellarwar/train.csv')
x_test_data = pd.read_csv('../input/heuristics-dl-interstellarwar/test.csv')
sample_train0 = x_train_data[x_train_data['High-Salary']==0]
sample_train1 = x_train_data[x_train_data['High-Salary']==1]
x_train_data = pd.concat([sample_train0,sample_train1.sample(n=sample_train0.shape[0])])
y_train = x_train_data['High-Salary']


#norm_x_train_data=(x_train_data.select_dtypes(include=['float64','int64'])-x_train_data.min(numeric_only=True))/(x_train_data.max(numeric_only=True)-x_train_data.min(numeric_only=True))
#norm_x_test_data=(x_test_data.select_dtypes(include=['float64','int64'])-x_test_data.min(numeric_only=True))/(x_test_data.max(numeric_only=True)-x_test_data.min(numeric_only=True))
correl = x_train_data.corr()['High-Salary'].sort_values()
correl.drop('High-Salary',inplace=True)
selected = correl.loc[correl>0.1]
attr_selected = list(selected.index)
norm_x_train_data = preprocessing.scale(x_train_data[attr_selected])
norm_x_test_data = preprocessing.scale(x_test_data[attr_selected])



In [3]:
model = keras.Sequential(
[
    layers.Dense(units=1024,activation='relu',input_shape=[len(attr_selected)]),
    layers.Dense(units=1024,activation='relu'),
    layers.Dense(units=1024,activation='relu'),
    layers.Dense(units=1024,activation='relu'),
    layers.Dense(units=1024,activation='relu'),
    layers.Dense(units=1,activation='sigmoid')
])
model.compile(
    optimizer = 'Adamax',
    loss = 'mae'
)
earlystop = EarlyStopping(min_delta=0.01,patience=10,restore_best_weights=True)
model.fit(
    x = norm_x_train_data,
    y = y_train,
    epochs = 200,
    batch_size = 256,
    callbacks=[earlystop]
)

2022-10-28 12:26:18.343892: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-10-28 12:26:18.542517: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
1/1 [==============================] - 1s 785ms/step - loss: 0.4997
Epoch 2/200
1/1 [==============================] - 0s 55ms/step - loss: 0.4427
Epoch 3/200
1/1 [==============================] - 0s 54ms/step - loss: 0.3553
Epoch 4/200
1/1 [==============================] - 0s 54ms/step - loss: 0.2972
Epoch 5/200
1/1 [==============================] - 0s 56ms/step - loss: 0.2777
Epoch 6/200
1/1 [==============================] - 0s 56ms/step - loss: 0.2633
Epoch 7/200
1/1 [==============================] - 0s 56ms/step - loss: 0.2491
Epoch 8/200
1/1 [==============================] - 0s 55ms/step - loss: 0.2383
Epoch 9/200
1/1 [==============================] - 0s 54ms/step - loss: 0.2265
Epoch 10/200
1/1 [==============================] - 0s 55ms/step - loss: 0.2189
Epoch 11/200
1/1 [==============================] - 0s 56ms/step - loss: 0.2290
Epoch 12/200
1/1 [==============================] - 0s 55ms/step - loss: 0.2091
Epoch 13/200
1/1 [==============================

In [4]:
y_pred = pd.Series(model.predict(norm_x_test_data).flatten()).rename('High-Salary')
submission = pd.DataFrame(x_test_data['ID']).join(y_pred).set_index(x_test_data['ID'])
submission.drop('ID', inplace=True, axis=1)
submission.loc[submission["High-Salary"]<0.5] = 0
submission.loc[submission["High-Salary"]>=0.5] = 1
submission = submission.astype(int)
submission.to_csv('submission.csv',index='False')
submission

,High-Salary
ID,
552848,1
226736,0
1029931,0
274730,0
1118166,0
...,...
852189,1
1240608,0
806319,1
